In [8]:
!pip install datasets

In [9]:
from transformers import AutoTokenizer, LlamaConfig, LlamaForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from transformers import DataCollatorForLanguageModeling
import torch

In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [18]:

# 2. Define a Mini LLaMA Config (simulate small pretraining)
config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,
    intermediate_size=2048,
    num_attention_heads=8,
    num_hidden_layers=4,
    max_position_embeddings=512,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

model = LlamaForCausalLM(config)

In [19]:

# 3. Load or Create Tiny Dataset (for demonstration)
texts = [
    "The robot learned to walk using reinforcement learning.",
    "Yoga improves flexibility and mental health.",
    "AI is transforming the future of technology and medicine."
]
dataset = Dataset.from_dict({"text": texts})

In [21]:
# 4. Tokenize Dataset
def tokenize(example):
    # Set the padding token to the EOS token if it's not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [22]:
# 5. Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 6. Training Arguments
training_args = TrainingArguments(
    output_dir="./llama-pretrain-demo",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=5,
    save_steps=10,
    save_total_limit=1,
    report_to="none"
)

In [23]:
# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)


<ipython-input-23-96956d39c49f>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
# 8. Pre-train the model
trainer.train()

# 9. Save model
model.save_pretrained("./llama-pretrain-demo")
tokenizer.save_pretrained("./llama-pretrain-demo")


Step,Training Loss
5,10.413200


('./llama-pretrain-demo/tokenizer_config.json',
 './llama-pretrain-demo/special_tokens_map.json',
 './llama-pretrain-demo/tokenizer.model',
 './llama-pretrain-demo/added_tokens.json',
 './llama-pretrain-demo/tokenizer.json')